In [1]:
# English tokenizer data
!spacy download en_core_web_sm

2024-06-21 02:56:47.439397: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-21 02:56:47.439457: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-21 02:56:47.440796: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-21 02:56:47.448015: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-21 02:56:48.365434: W tensorflow/compiler/tf2

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

from dataset import Flickr8kDataset
from dataset import Collate
import os
import torchvision.transforms as transforms

2024-06-21 02:56:57.976250: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-21 02:56:57.976324: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-21 02:56:57.977701: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-21 02:56:57.985348: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-21 02:56:58.904703: W tensorflow/compiler/tf2

In [3]:
inception = models.inception_v3(weights='IMAGENET1K_V1', aux_logits = True)

In [4]:
modules = list(inception.children())[:-3]
modules.remove(modules[15])
incept = nn.Sequential(*modules)

In [5]:
resnet = models.resnet50(weights='ResNet50_Weights.DEFAULT')

In [6]:
modules = list(resnet.children())[:-2]

In [7]:
modules
resnet = nn.Sequential(*modules)

In [8]:
# Data transformations
transform = transforms.Compose(
    [
        transforms.Resize((512,512)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

# Dataset loading
dataset_path = "./dataset/"
root = os.path.join(dataset_path, 'Images')
annotations = os.path.join(dataset_path, 'captions.txt')
freq_threshold = 6
dataset = Flickr8kDataset(root, annotations, transform, freq_threshold)

In [9]:
a, b  = dataset.__getitem__(0)

In [10]:
c = a.unsqueeze(0)
c.shape

torch.Size([1, 3, 512, 512])

In [11]:
test_batch = torch.rand(2, 3, 512, 512)
test_batch.shape

torch.Size([2, 3, 512, 512])

In [12]:
output1 = resnet(test_batch)
output1.shape
#mod = models.resnet50(weights='ResNet50_Weights.DEFAULT')
#d = mod(test_batch)

torch.Size([2, 2048, 16, 16])

In [13]:
output2 = incept(test_batch)
output2.shape

torch.Size([2, 2048, 14, 14])

In [14]:
b.shape

torch.Size([20])

In [15]:
test_y = torch.randint(low = 0, high = 100, size = (2, 20))

In [16]:
embed = nn.Embedding(100, 50)
e = embed(test_y)

In [17]:
e.shape

torch.Size([2, 20, 50])

In [18]:
from image_captioning_from_scratch import CNNtoRNN
import torch

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
embed_size = 512
decoder_dim = 512
attention_dim = 256
vocab_size = len(dataset.vocab)
p = 0.5

model = CNNtoRNN(embed_size, decoder_dim, attention_dim, vocab_size, p).to(device)

In [20]:
test_output = model(test_batch.to(device), test_y.to(device))

In [24]:
test_output

tensor([[[ 0.1082, -0.0967, -0.1160,  ..., -0.0341, -0.0346, -0.0107],
         [ 0.0877, -0.0701, -0.0213,  ..., -0.0932, -0.0588,  0.2540],
         [-0.1103, -0.0972, -0.3115,  ..., -0.1512, -0.1922,  0.2918],
         ...,
         [-0.2918, -0.0782,  0.2177,  ..., -0.1686, -0.1274,  0.2419],
         [-0.0234, -0.2008,  0.1268,  ...,  0.0716,  0.0150,  0.2788],
         [-0.1476, -0.3081, -0.0360,  ..., -0.0327, -0.1804,  0.0091]],

        [[ 0.0401, -0.1147,  0.1005,  ..., -0.1329, -0.0281,  0.2134],
         [ 0.0370,  0.1097,  0.2531,  ...,  0.2245, -0.0519,  0.2867],
         [-0.1237, -0.1456,  0.2334,  ..., -0.0754, -0.0502,  0.2079],
         ...,
         [ 0.2183,  0.0809,  0.1118,  ...,  0.1393, -0.2648, -0.1994],
         [ 0.0306,  0.3158,  0.1289,  ..., -0.0486, -0.2644,  0.2190],
         [ 0.0172,  0.1023,  0.2340,  ..., -0.0110,  0.1199,  0.2362]]],
       grad_fn=<CopySlices>)